## AE: AutoEncoder
- **비지도학습** 인공지능
- 입력 데이터의 특징점을 효율적으로 찾는 것
- AE의 출력 데이터는 입력한 데이터에 비해 정보의 차원이 줄기 때문에 잡음 성분을 제거하는 용도로도 사용

#### AE의 원리
- 데이터 압축
- 저차원화를 통한 데이터 관계 관찰
- 배경 잡음 억제(denoising)

--- 


- 주성분 분석(PCA)로 처리하는 일차원 데이터 처리 방식을 딥러닝 방식으로 확장한 것
- AE는 신경망으로 특징점을 추출하기 때문에 **데이터 구성이 복잡하거나 데이터가 대량인 경우**에 주성분 분석 방법보다 효과적

**부호화**:
- 입력 계층에서 들어온 다차원 데이터는 차원을 줄이는 은닉계층으로 들어감
- 은닉계층의 출력이 곧 부호화 결과

**복호화**
- 은닉계층에서 출력한 부호화 결과는 출력 계층으로 들어감
- 이때 출력 계층의 노드수는 은닉계층의 노드 수보다 많음
- 즉, 더 높은 차원의 데이터로 되돌아감


부호화 -> 복호화 (Downsampling-> upsampling)


## 완전 연결 계층을 이용한 AE

In [8]:
from keras import layers, models

class AE(models.Model):
    def __init__(self, x_nodes=784, z_dim=36):
        x_shape = (x_nodes,)# 입력 노드의 수(입력노드 수로 구성된 튜플)
        x = layers.Input(shape=x_shape)#입력계층
        z = layers.Dense(z_dim, activation='relu')(x) #은닉계층
        y = layers.Dense(x_nodes, activation='sigmoid')(z) # 출력계층

        super().__init__(x, y)

        self.x = x
        self.z = z
        self.z_dim = z_dim

        # Encoder, Decoder ??

        self.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    def Encoder(self):
        return models.Model(self.x, self.z)

    def Decoder(self):
        z_shape = (self.z_dim,)
        z = layers.Input(shape=z_shape)
        y_layer = self.layers[-1]
        y = y_layer(z)
        return models.Model(z, y)

In [10]:
# 데이터 준비
from keras.datasets import mnist
import numpy as np

# label이 따로 필요없음
(X_train, _), (X_test, _) = mnist.load_data()

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
print(X_train.shape)
print(X_test.shape)

(60000, 784)
(10000, 784)


In [16]:
def main():
    x_nodes = 784
    z_dim = 36

    autoencoder = AE(x_nodes, z_dim)

    history = autoencoder.fit(X_train, X_train,
                              epochs=10,
                              batch_size=256,
                              shuffle=True,
                              validation_data=(X_test, X_test))

#     plot_acc(history, '(a) 학습 경과에 따른 정확도 변화 추이')
#     plt.show()
#     plot_loss(history, '(b) 학습 경과에 따른 손실값 변화 추이')
#     plt.show()

#     show_ae(autoencoder)
#     plt.show()

In [17]:
if __name__ == '__main__':
    main()

Epoch 1/10
235/235 [==============================] - 5s 18ms/step - loss: 0.6923 - accuracy: 9.4317e-04 - val_loss: 0.6922 - val_accuracy: 0.0013
Epoch 2/10
235/235 [==============================] - 6s 24ms/step - loss: 0.6921 - accuracy: 9.9472e-04 - val_loss: 0.6920 - val_accuracy: 0.0013
Epoch 3/10
235/235 [==============================] - 6s 25ms/step - loss: 0.6920 - accuracy: 0.0012 - val_loss: 0.6918 - val_accuracy: 0.0012
Epoch 4/10
235/235 [==============================] - 5s 20ms/step - loss: 0.6918 - accuracy: 9.9382e-04 - val_loss: 0.6917 - val_accuracy: 0.0011
Epoch 5/10
235/235 [==============================] - 4s 17ms/step - loss: 0.6916 - accuracy: 0.0010 - val_loss: 0.6915 - val_accuracy: 0.0011loss: 0.6916 - accuracy: 0.00
Epoch 6/10
235/235 [==============================] - 5s 21ms/step - loss: 0.6915 - accuracy: 0.0011 - val_loss: 0.6913 - val_accuracy: 0.0011
Epoch 7/10
235/235 [==============================] - 4s 18ms/step - loss: 0.6913 - accuracy: 0.0011 

## 합성곱 계층을 이용한 AE 구현

In [1]:
from keras import layers, models

def Conv2D(filters, kernel_size, padding = 'same', activation = 'relu'):
    return layers.Conv2D(filters, kernel_size, padding= padding, activation = activation)

In [2]:
class AE(models.Model):
    def __init__(self, org_shape = (1, 28, 28)):
        # Input
        original = layers.Input(shape = org_shape)
        
        # encoding-1
        x = Conv2D(4, (3, 3))(original)
        x = layers.MaxPooling2D((2,2), padding='same')(x)
        
        # encoding-2
        x = Conv2D(8, (3,3))(x)
        x = layers.MaxPooling2D((2,2), padding= 'same')(x)
        
        # encoding-3: encoding output: 7x7 pixels
        z = Conv2D(1, (7,7))(x)
        
        # decoding-1
        y = Conv2D(16, (7,7))(z)
        y = layers.UpSampling2D((2,2))(y)
        
        # decoding-2
        y = Conv2D(8, (3,3))(z)
        y = layers.UpSampling2D((2,2))(y)
        
        # decoding-3
        y = Conv2D(4, (3,3))(y)
        
        # decoding & Output
        decoded = Conv2D(1, (3,3), activation = 'sigmoid')(y)
        
        super().__init__(original, decoded)
        self.compile(optimizer = 'adadelta', loss= 'binary_crossentropy', metrics = ['accuracy'])